In [1]:
using TensorFlow
using Distributions
using MLDataUtils
using SwiftObjectStores
using Iterators
using ColoringNames
using MappedArrays
using MLLabelUtils
using StaticArrays
using Base.Test

Base.one{S<:AbstractString}(::Type{S}) = ""

In [2]:
serv=SwiftService()
train_raw = get_file(serv, "color", "monroe/train.csv") do fh
    readdlm(fh,'\t')
end

1523108×4 Array{Any,2}:
 "acid green"  0.344113  0.855319  0.921569
 "acid green"  0.281905  0.744681  0.921569
 "acid green"  0.347551  0.921397  0.898039
 "acid green"  0.311209  0.900398  0.984314
 "acid green"  0.312085  0.992095  0.992157
 "acid green"  0.22766   0.987395  0.933333
 "acid green"  0.288462  0.466368  0.87451 
 "acid green"  0.276936  0.804878  0.964706
 "acid green"  0.325359  0.95      0.862745
 "acid green"  0.385201  0.971774  0.972549
 "acid green"  0.212885  0.955823  0.976471
 "acid green"  0.346405  0.6       1.0     
 "acid green"  0.238397  0.42246   0.733333
 ⋮                                         
 "yuck"        0.1621    0.421965  0.678431
 "yuck"        0.153333  0.722543  0.678431
 "yuck"        0.201754  0.256757  0.580392
 "yuck"        0.264463  0.620513  0.764706
 "yuck"        0.144608  0.790698  0.67451 
 "yuck"        0.178125  0.97561   0.643137
 "yuck"        0.166667  0.350962  0.815686
 "yuck"        0.174603  0.640244  0.643137
 "yuck" 

In [3]:
tokenizer = morpheme_tokenizer("../data/replacement_rules.csv")

(::tokenize) (generic function with 1 method)

In [4]:
keep_inds = shuffle(1:size(train_raw, 1))[1:100_000]

labels = convert(Vector{String}, train_raw[keep_inds,1]); #TODO use all data
hsv_data = convert(Matrix{Float64}, train_raw[keep_inds,2:end]); #TODO use all data


100000×3 Array{Float64,2}:
 0.539141  0.564103  0.917647
 0.314928  0.831633  0.768627
 0.22619   0.375     0.439216
 0.243842  0.890351  0.894118
 0.572304  0.559671  0.952941
 0.677011  0.884146  0.643137
 0.830128  0.327044  0.623529
 0.652344  0.549356  0.913725
 0.943452  0.823529  0.266667
 0.948649  0.731225  0.992157
 0.450549  0.654676  0.545098
 0.422101  0.973545  0.741176
 0.674954  0.780172  0.909804
 ⋮                           
 0.732906  0.995745  0.921569
 0.928     0.748503  0.654902
 0.295198  0.967213  0.956863
 0.235727  0.79386   0.894118
 0.690476  0.573276  0.909804
 0.238215  0.846154  0.917647
 0.947917  0.898876  0.698039
 0.387067  0.753086  0.952941
 0.18107   0.707424  0.898039
 0.204981  0.386667  0.882353
 0.728797  0.698347  0.94902 
 0.382456  0.88785   0.419608

In [5]:
tokenised_labels = tokenizer.(labels)
all_tokens = reduce(union, tokenised_labels)
encoding = labelenc(all_tokens)
label_inds = map(x->label2ind.(x, Scalar(encoding)), tokenised_labels)

100000-element Array{Array{Int64,1},1}:
 [1]      
 [2]      
 [3]      
 [4,2]    
 [5]      
 [1]      
 [6,7]    
 [1]      
 [8,7]    
 [9]      
 [10,2]   
 [2]      
 [1]      
 ⋮        
 [7]      
 [33]     
 [2]      
 [2]      
 [1]      
 [2,13,17]
 [8,9]    
 [2]      
 [17,18,2]
 [19]     
 [7]      
 [8,2]    

In [6]:
function Base.rpad{T}(xs::Vector{T}, n::Integer, p::T=zero(T))
    sizehint!(xs, n)
    while length(xs)<n
        push!(xs, p)
    end
    xs
end

"Creates a matrix where each column is one of the vectors from `xss`"
function rpad_to_matrix{T}(xss::Vector{Vector{T}}, n_rows::T=maximum(length.(xss)), p::T=zero(T))
    n_cols = length(xss) 
    ret = fill(p, (n_rows, n_cols))
    for (cc, xs) in enumerate(xss)
        for (rr, x) in enumerate(xs)
            @inbounds ret[rr, cc] = x
        end
    end
    ret
end

@testset "padding" begin
    @test rpad([1,2,3],5) == [1,2,3,0,0]
    @test rpad([1,2,3],2) == [1,2,3]
    @test rpad([1,2,3],5, 8 ) == [1,2,3,8,8]
    
    @test rpad_to_matrix([[1,2], [3],Int[],[2]])==[
                                                    1 3 0 2
                                                    2 0 0 0
    ]
end;

Test Summary: | Pass  Total


In [9]:
padded_labels = rpad_to_matrix(label_inds, 5)

5×100000 Array{Int64,2}:
 1  2  3  4  5  1  6  1  8  9  10  2  …  33  2  2  1   2  8  2  17  19  7  8
 0  0  0  2  0  0  7  0  7  0   2  0      0  0  0  0  13  9  0  18   0  0  2
 0  0  0  0  0  0  0  0  0  0   0  0      0  0  0  0  17  0  0   2   0  0  0
 0  0  0  0  0  0  0  0  0  0   0  0      0  0  0  0   0  0  0   0   0  0  0
 0  0  0  0  0  0  0  0  0  0   0  0      0  0  0  0   0  0  0   0   0  0  0

In [13]:
learning_rate = 0.05

n_input = 3 # HSV
@show n_steps, batch_size = size(padded_labels)
hidden_layer_size = 256
n_classes = nlabel(encoding)+1



(n_steps,batch_size) = size(padded_labels) = (5,100000)


312

In [14]:
sess = Session(Graph())
X = placeholder(Float32, shape=[batch_size, n_input]; name="X_HSVs")
Term_obs_s = placeholder(Int32, shape=[n_steps, batch_size]; name="Term_obs_s")


Xs = fill(X, n_steps)


Hs, states = nn.rnn(nn.rnn_cell.LSTMCell(hidden_layer_size), Xs; dtype=Float32)#, sequence_length=n_steps);

variable_scope("fromColor", initializer=Normal(0, .1)) do
    
    
    global W1 = get_variable("weights1", [hidden_layer_size, n_classes], Float32)
    global B1 = get_variable("bias1", [n_classes], Float32)

    global Ls =  [H*W1+B1 for H in Hs]
    
    #global Ls =  [TensorFlow.matmul(H, W1; name="Ls$ii") for (ii,H) in enumerate(Hs)]
end;


#LL = concat(3, expand_dims(.(Ls, Scalar(3))); name="Stack_Logits") #Stack
#LL = concat(2, [expand_dims(L, 2; name="Le$ii") for (ii,L) in enumerate(Ls)]; name="Stack_Logits") #Stack
LL = concat(0, Ls; name="Stack_Logits") #Stack

TT = reshape(Term_obs_s, [n_steps*batch_size]; name="Stack_Term_obs_s")

Term_preds_onehots = nn.softmax(LL; name="Term_preds_onehots")
Term_preds_s = indmax(Term_preds_onehots, 2)-1

Term_obs_onehots = one_hot(TT, n_classes)
costs  = (reduce_sum(Term_obs_onehots.*log(Term_preds_onehots), reduction_indices=[1]))

#costs = nn.sparse_softmax_cross_entropy_with_logits(LL, TT+1)
cost = reduce_mean(-costs) #cross entropy



optimizer = train.minimize(train.AdamOptimizer(learning_rate), cost)

run(sess, initialize_all_variables())



In [ ]:
global Term_preds_s_o
global cost_o
global optimizer_o
for i in 1:1000
    Term_preds_s_o,  cost_o, optimizer_o = run(sess, [Term_preds_s, cost, optimizer], Dict(X=>hsv_data, Term_obs_s=>padded_labels))
    @show (i, cost_o)
    @show (i, unique(Term_preds_s_o))
end



In [ ]:
ind2label.(Term_preds_s_o[Term_preds_s_o.>0], Scalar(encoding)) |> print

Nope

In [ ]:
ind2label.(Term_preds_s_o, Scalar(encoding)) |> print

In [13]:
x = transpose(X, Int32.([1, 3, 2].-1)) # Permuting batch_size and n_steps. (the -1 is to use 0 based indexing)
x = reshape(x, [n_steps*batch_size, n_input]) # Reshaping to (n_steps*batch_size, n_input)
x = split(1, n_steps, x) # Split to get a list of 'n_steps' tensors of shape (batch_size, n_input)
@show get_shape.(x);

get_shape.(x) = TensorFlow.ShapeInference.TensorShape[TensorShape[1524, 3],TensorShape[1524, 3],TensorShape[1524, 3],TensorShape[1524, 3],TensorShape[1524, 3]]


In [15]:
sess = Session(Graph())

embedding_dim = 16
hidden_layer_size = 128
variable_scope("ToColor", initializer=Normal(0, .1)) do
    global hsv_obs = placeholder(Float64)
    global word_ids = placeholder(Int64)
    global Ts = get_variable("TokenEmbeddings",  [length(tokens), embedding_dim], Float32)
    global X_embs = nn.embedding_lookup(Ts,  X_ids);
    global SoWe = reduce_sum(X_embs; reduction_indices=1, keep_dims=true); #Sum up however many embeddings we have
    
    global W1 = get_variable("weights1", [embedding_dim, hidden_layer_size], Float32)
    global B1 = get_variable("bias1", [hidden_layer_size,], Float32)    
    
    global Wo = get_variable("weightsout", [hidden_layer_size, output_dim], Float32)
    global Bo = get_variable("biasout", [output_dim,], Float32)    
    
    global H = nn.sigmoid(SoWe*W1 + B1)
    global Y = nn.tanh(H*Wo + Bo)
    global Loss = reduce_sum((Y - Y_obs)^2)

end;

LoadError: UndefVarError: tokens not defined

In [51]:
methods(nn.sparse_softmax_cross_entropy_with_logits)

# 1 method for generic function "sparse_softmax_cross_entropy_with_logits":
sparse_softmax_cross_entropy_with_logits(logits, labels; name) at /home/ubuntu/.julia/v0.5/TensorFlow/src/ops/nn.jl:201

In [52]:
using Distributions
using TensorFlow

# Generate some synthetic data
x = randn(100, 50)
w = randn(50, 10)
y_prob = exp(x*w)
y_prob ./= sum(y_prob,2)

function draw(probs)
    y = zeros(size(probs))
    for i in 1:size(probs, 1)
        idx = rand(Categorical(probs[i, :]))
        y[i, idx] = 1
    end
    return y
end

y = draw(y_prob)

# Build the model
sess = Session(Graph())
X = placeholder(Float64)
Y_obs = placeholder(Float64)
Y_obs_lbl = indmax(Y_obs, 2)
variable_scope("logisitic_model", initializer=Normal(0, .001)) do
    global W = get_variable("weights", [50, 10], Float64)
    global B = get_variable("bias", [10], Float64)
end

L = X*W + B
Y=nn.softmax(L)
#costs = log(Y).*Y_obs #Dense (Orginal) way
costs = nn.sparse_softmax_cross_entropy_with_logits(L, Y_obs_lbl+1) #sparse way
Loss = -reduce_sum(costs)
optimizer = train.AdamOptimizer()
minimize_op = train.minimize(optimizer, Loss)
saver = train.Saver()
# Run training
run(sess, initialize_all_variables())
cur_loss, = run(sess, [Loss], Dict(X=>x, Y_obs=>y))

1-element Array{Float64,1}:
 -230.249

In [43]:
methods(indmax)

# 4 methods for generic function "indmax":
indmax(f::Array{Base.Pkg.Resolve.MaxSum.FieldValues.FieldValue,1}) at pkg/resolve/fieldvalue.jl:74
indmax(A::SparseMatrixCSC) at sparse/sparsematrix.jl:2073
indmax(n::TensorFlow.AbstractTensor, dim) at /home/ubuntu/.julia/v0.5/TensorFlow/src/ops/math.jl:68
indmax(a) at array.jl:1281